Se importan las bibliotecas necesarias para el análisis de datos, manipulación de datos, y creación de modelos estadísticos.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

Se carga el archivo de Excel en un DataFrame de pandas.

In [2]:
df = pd.read_excel('Databases/1. Generales - Original.xlsx')

Se renombra una columna para evitar problemas con el nombre.

In [3]:
df = df.rename(columns={'autopercep_izq-der': 'autopercep_izq_der'})

Se definen las variables dependientes (la variable que se quiere predecir) y las variables independientes (las que influyen en la dependiente). Al ser un modelo ordinal, la variable dependiente debe ser ordinal, o sea, debe tener categorías que pueden ser ordenadas jerárquicamente, como calificaciones (por ejemplo, "mala", "regular", "buena") o niveles de satisfacción (por ejemplo, "insatisfecho", "neutral", "satisfecho") 

In [4]:
# Variables a explicar.
Variables_Dependientes = ['cercania_Massa', 
                          #'cercania_Milei', 
                          #'cercania_Bregman', 
                          #'cercania_Bullrich', 
                          #'cercania_Schiaretti'
                          ]

# Variables que influyen en las otras.
Variables_Todas = [
                            # 'genero',
                            # 'edad',
                            # 'nacionalidad',
                            # 'provincia',
                            'e_social',
                            # 'niv_educativo',
                            'voto_2019',
                            # 'voto_PASO_2023',
                            'candidato_PASO_2023',
                            # 'autopercep_izq_der',
                            # 'autopercep_conpro',
                            # 'autopercep_perantiper',
                            'indice_positividad',
                            'massa_ip_izqder',
                            'massa_ip_conpro',
                            # 'bullrich_ip_izqder',
                            # 'bullrich_ip_conpro',
                            # 'schiaretti_ip_izqder',
                            # 'schiaretti_ip_conpro',
                            # 'milei_ip_izqder',
                            # 'milei_ip_conpro',
                            # 'bregman_ip_izqder',
                            # 'bregman_ip_conpro',
                            # 'indice_progresismo',
                            # 'indice_conservadurismo'
                            ]

Se crea una lista de variables que se utilizarán para el análisis de un candidato específico (Massa).

In [5]:
Variables_Massa = [
    'voto_2019',
    'autopercep_perantiper',
    'massa_ip_conpro',
    #'indice_conservadurismo',
    #'massa_ip_izqder',
    #'edad',
    'autopercep_conpro',
    #'indice_progresismo',
    #'autopercep_izq_der',
    #'indice_positividad',
    'candidato_PASO_2023',
    #'schiaretti_ip_conpro',
    #'schiaretti_ip_izqder'
]

In [6]:
Variables_Independientes = Variables_Massa

Se asignan las variables independientes a X y la variable dependiente a y.

In [7]:
X = df[Variables_Independientes]
y = df[Variables_Dependientes]

Se convierte cualquier variable categórica en variables numéricas usando LabelEncoder. Esto es necesario porque muchos modelos estadísticos requieren entradas numéricas.

In [8]:
# Convert categorical independent variables to numerical using LabelEncoder.
for Column in Variables_Independientes:
    if X[Column].dtype == 'object':
        X[Column] = LabelEncoder().fit_transform(X[Column])

C:\Users\tomas\AppData\Local\Temp\ipykernel_9204\3576202961.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[Column] = LabelEncoder().fit_transform(X[Column])
C:\Users\tomas\AppData\Local\Temp\ipykernel_9204\3576202961.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[Column] = LabelEncoder().fit_transform(X[Column])


Los datos se dividen en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%) para poder evaluar el modelo posteriormente.

In [9]:
# Split the data into training and testing sets.
X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size=0.2, random_state=42)

Se ajusta un modelo de regresión logística ordinal a los datos de entrenamiento. Esto es apropiado ya que la variable dependiente es ordinal.

In [10]:
# Ajustar el modelo de regresión logística ordinal sin añadir la constante manualmente.
Ordinal_Model = OrderedModel(y_Train, X_Train, distr='logit')
Ordinal_Result = Ordinal_Model.fit()

C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\optimizer.py:737: RuntimeWarning: Maximum number of iterations has been exceeded.
  retvals = optimize.fmin(f, start_params, args=fargs, xtol=xtol,
C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Se imprime un resumen del modelo no lineal ajustado para revisar los coeficientes y la significancia estadística.

In [11]:
# Resumen del modelo ajustado para verificar los coeficientes de los términos.
print(Ordinal_Result.summary())

                             OrderedModel Results                             
Dep. Variable:         cercania_Massa   Log-Likelihood:                -2721.9
Model:                   OrderedModel   AIC:                             5462.
Method:            Maximum Likelihood   BIC:                             5513.
Date:                Sun, 20 Oct 2024                                         
Time:                        15:12:04                                         
No. Observations:                2271                                         
Df Residuals:                    2262                                         
Df Model:                           5                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
voto_2019                -0.1402      0.019     -7.561      0.000      -0.177      -0.104
autopercep_perantip

Se realizan predicciones sobre el conjunto de prueba utilizando el modelo ajustado.

In [12]:
# Realizar predicciones en el set de prueba.
Predictions = Ordinal_Result.predict(X_Test)

__1. Verificar la linealidad del logit.__

----

Para verificar la linealidad entre las variables independientes y el logit, se pueden agregar términos cuadráticos y observar si estos son significativos. Verificá si el término cuadrático (edad_sq) es significativo (p-valor < 0.05). Si es significativo, puede indicar que la relación no es lineal.

In [13]:
# Verificar la linealidad del logit para cada variable independiente.
for Variable in Variables_Independientes:

    # Agregar el término cuadrático.
    X_Train[f'{Variable}_sq'] = X_Train[Variable] ** 2  

    # Ajustar el modelo ordinal con el término cuadrático.
    Ordinal_Model = OrderedModel(y_Train, X_Train[[Variable, f'{Variable}_sq']], distr='logit')
    Ordinal_Result = Ordinal_Model.fit(disp=0)  # 'disp=0' oculta la salida durante el ajuste.

    # Extraer el p-valor del término cuadrático.
    p_value = Ordinal_Result.pvalues[f'{Variable}_sq']

    # Imprimir solo el p-valor.
    print(f"P-value for {Variable} (quadratic term): {p_value:.4f}")  # Format p-value to 4 decimal places.

P-value for voto_2019 (quadratic term): 0.0000


C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


P-value for autopercep_perantiper (quadratic term): 0.2590


C:\Users\tomas\AppData\Roaming\Python\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


P-value for massa_ip_conpro (quadratic term): 0.0000
P-value for autopercep_conpro (quadratic term): 0.0161
P-value for candidato_PASO_2023 (quadratic term): 0.0153


__2. Verificar la proporcionalidad de las odds.__

-----

Se puede usar el test de Brant o el test de proporcionalidad de odds. Este test verifica si los coeficientes son constantes a través de los diferentes niveles de la variable dependiente.

In [14]:
# Realizar el test de Brant.
from ordinality import BrantTest

brant_test = BrantTest(Ordinal_Model)
brant_test.summary()

ModuleNotFoundError: No module named 'ordinality'

In [15]:
pip install ordinality

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ordinality (from versions: none)
ERROR: No matching distribution found for ordinality


__3. Independencia de las observaciones.__

------

Para verificar la independencia de las observaciones, se puede revisar si hay duplicados o agrupaciones en los datos. En el contexto de encuestas, esto suele ser asumido, pero es bueno verificar:

In [143]:
# Revisar duplicados en el DataFrame
print(df.duplicated().sum())

0
